In [13]:
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import requests
import scipy
from PIL import Image
from torch.utils.data import Dataset, Subset, random_split, DataLoader
from torchvision import transforms
from tqdm.auto import tqdm
import os

## Dataset Class

In [ ]:
class FlowerDataset(Dataset):
    def __inti__(self, root_dir, transform = None):

        self.root_dir = root_dir

        self.transform = transform

        self.img_dir = os.path.join(self.root_dir, "jpg")

        self.labels = os.path.join(self.root_dir, "labels")

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):

        image = self.retrieve_image(index)

        if self.transform is not None:
            image = self.transform(image)

            label = self.labels(index)

        return image, label
    
    def retrieve_image(self,index):
        img_name = f"image_{index + 1:05d}"

## CNN Architecture

In [10]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # First Convolutional Block
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        # Second Convolutional Block
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        # Third Convolutional Block
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        # Flatten Flayer 
        self.flatten = nn.Flatten()

        # Fully-connected Layer
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 15)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

## Model Instantiation

In [11]:
model = SimpleCNN()

## Loss Function and Optimizer

In [12]:
loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr = 0.0005, weight_decay = 0.0005)